
# 📓 05 - RAG com LangChain carregando arquivos reais (.txt, .md, .csv)

Neste notebook vamos:  
✅ Carregar documentos reais usando loaders do LangChain.  
✅ Indexar e buscar usando FAISS.  
✅ Criar um pipeline RAG com `RetrievalQAChain`.  


In [ ]:

# !pip install langchain langchain-community langchain-openai sentence-transformers faiss-cpu
# !pip install python-dotenv pandas


In [1]:

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, CSVLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA


In [2]:

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")



## 📝 1️⃣ Carregando arquivos reais (.txt, .md, .csv)


In [3]:
from langchain.schema import Document
import pandas as pd

In [10]:
# Só carrega TXT e CSV
txt_loader = TextLoader(file_path="../data/exemplo.txt", encoding="utf-8")
txt_docs = txt_loader.load()

# csv_loader = CSVLoader(file_path="../data/exemplo.csv")
# csv_docs = csv_loader.load()

# df = pd.read_csv("../data/exemplo.csv")
# table_md = df.to_markdown(index=False)
# doc_csv = Document(page_content=table_md, metadata={"source": "../data/exemplo.csv"})

md_loader = UnstructuredMarkdownLoader(file_path="../data/exemplo.md")
md_docs = md_loader.load()

all_docs = txt_docs + md_docs


## ✂️ 2️⃣ Chunking dos documentos


In [11]:

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs_split = splitter.split_documents(all_docs)

print(f"🔹 {len(docs_split)} chunks criados ao todo")


🔹 3 chunks criados ao todo



## 💾 3️⃣ Indexar no FAISS


In [12]:

embeddings = OpenAIEmbeddings(api_key=openai_api_key)
vectorstore = FAISS.from_documents(docs_split, embeddings)

print("🔹 FAISS indexado com documentos reais!")


🔹 FAISS indexado com documentos reais!



## 🤖 4️⃣ Criar pipeline RAG com LangChain


In [13]:

llm = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo")  #"gpt-4o")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

query = "O que está descrito nesses arquivos?"
result = qa_chain({"query": query})

print("🔹 Resposta final:", result["result"])

print("\n🔹 Documentos recuperados:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source', 'desconhecido')}")
    print("  Trecho:", doc.page_content[:100], "...")


🔹 Resposta final: Os arquivos descrevem o funcionamento do Retrieval-Augmented Generation (RAG), que combina recuperação de informações e geração de linguagem natural. O RAG busca dados em fontes externas para fornecer respostas mais relevantes e atualizadas. Além disso, destacam as vantagens do RAG, como respostas mais completas e baseado em dados dinâmicos.

🔹 Documentos recuperados:
- ../data/exemplo.txt
  Trecho: Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natur ...
- ../data/exemplo.md
  Trecho: Vantagens do RAG

Respostas mais completas

Baseado em dados dinâmicos ...



## 🧐 Conclusão

✅ Pipeline RAG agora carrega arquivos reais e gera respostas baseadas neles.  
✅ Pronto para evoluir para outros bancos vetoriais ou fluxos mais complexos. 🚀
